In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import transforms
import ast #for converting string dictionary to dictionary
import cv2
from skimage import io
from skimage.draw import polygon2mask

In [3]:
path = '/Users/makhanov/Yandex.Disk.localized/NU-PhD/Fall2020/ComputerVision/project/datasets/Covid_Lung_segments0_400_csv.csv'
dataset = pd.read_csv(path)

In [ ]:
dataset['region_id']==0

In [192]:
filtered_target = dataset[(dataset['filename']=='Covid (77).png') & (dataset['region_id']==0)]
filtered_target['region_shape_attributes']

152    {"name":"polygon","all_points_x":[106,95,84,73...
Name: region_shape_attributes, dtype: object

In [4]:
#region_id: left_lung - 0, right_lung - 1
#region_attributes: COVID - infected, non-COVID - healthy

In [5]:
region_shape_attributes = dataset['region_shape_attributes']

In [6]:
for i in range(len(region_shape_attributes)):
    filename = dataset['filename'][i]
    
    region_attributes = ast.literal_eval(dataset['region_attributes'][i])
    if region_attributes['COVID'] == 'infected':
        label = 'COVID'
    else:
        label = 'non-COVID'
        
    lung = dataset['region_id'][i] #region_id: left_lung - 0, right_lung - 1
    
    region = ast.literal_eval(region_shape_attributes[i])
    x = region['all_points_x']
    y = region['all_points_y']
        

ValueError: malformed node or string: <_ast.Name object at 0x7fa156e64190>

In [7]:
str1 = region_shape_attributes[152]

In [8]:
str1

'{"name":"polygon","all_points_x":[106,95,84,73,66,56,45,34,21,10,6,3,0,0,6,10,18,27,38,49,65,79,90,101,111,120,126,130,131,129,125,120,114,106,101,98,97,95,96,102,109,113,116,125,131,137,146,155,165,171,171,169,163,158,150,140,133,122],"all_points_y":[17,16,14,11,10,13,16,23,33,47,56,64,73,139,155,167,178,187,194,197,198,195,191,184,177,170,163,156,152,147,142,139,135,130,127,122,115,108,102,95,90,85,80,74,67,61,55,48,38,32,28,24,18,17,16,18,20,19]}'

In [9]:
import ast 
res = ast.literal_eval(str1)

In [10]:
res.keys()

dict_keys(['name', 'all_points_x', 'all_points_y'])

In [11]:
x = res['all_points_x']
y = res['all_points_y']

In [16]:
image_path = '/Users/makhanov/Yandex.Disk.localized/NU-PhD/Fall2020/ComputerVision/project/datasets/data_for_segmentation/Covid (77).png'
# image = cv2.imread(image_path)
img = cv2.imread(image_path, 0)

In [18]:
img.shape

(234, 319)

In [23]:
pairs = []
for i in range(len(x)):
    pairs.append([x[i], y[i]])
    
print(pairs)


[[106, 17], [95, 16], [84, 14], [73, 11], [66, 10], [56, 13], [45, 16], [34, 23], [21, 33], [10, 47], [6, 56], [3, 64], [0, 73], [0, 139], [6, 155], [10, 167], [18, 178], [27, 187], [38, 194], [49, 197], [65, 198], [79, 195], [90, 191], [101, 184], [111, 177], [120, 170], [126, 163], [130, 156], [131, 152], [129, 147], [125, 142], [120, 139], [114, 135], [106, 130], [101, 127], [98, 122], [97, 115], [95, 108], [96, 102], [102, 95], [109, 90], [113, 85], [116, 80], [125, 74], [131, 67], [137, 61], [146, 55], [155, 48], [165, 38], [171, 32], [171, 28], [169, 24], [163, 18], [158, 17], [150, 16], [140, 18], [133, 20], [122, 19]]


In [56]:
mask = np.zeros((img.shape[0], img.shape[1]))

cv2.fillConvexPoly(mask, np.array(pairs), 1)
mask = mask.astype(np.bool)

out = np.zeros_like(img)
out[mask] = img[mask]

out[out > 0] = 255

In [ ]:
# cv2.imshow('Extracted Image', out)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [57]:
cv2.imwrite('output.png', out)

True

In [26]:
# fig, ax = plt.figure()
# # base = plt.gca().transData
# # rot = transforms.Affine2D().rotate_deg(20)
# fig.
# plt.rcParams['axes.facecolor']='black'
# plt.fill(x, y, 'white')
# # plt.xticks(rotation=45)
# plt.
# plt.show()
# #need to flip

In [23]:
# mask = polygon2mask(image.shape, (x,y))

In [170]:
test_img_path = '/Users/makhanov/Yandex.Disk.localized/NU-PhD/Fall2020/ComputerVision/project/coding/left_mask77.png'
test_img = cv2.imread(test_img_path, 0)

In [171]:
x, y = img.shape
resized_test_img = cv2.resize(test_img, (y,x), interpolation = cv2.INTER_AREA)

In [172]:
resized_test_img[resized_test_img>0]=255

In [168]:
def iou_numpy(outputs, labels):
    mask = 255
    intersections = 0
    h, w = outputs.shape
    for i in range(h):
        for j in range(w):
            if labels[i][j]==mask:
                if labels[i][j]==outputs[i][j]:
                    intersections += 1

    test = outputs[:]+labels[:]
    union = test[test>0]

#     print((intersections), len(union))
    iou = (intersections)/ float(len(union))
    
    return iou
    

In [173]:
print(iou_numpy(resized_test_img, out))

0.8568983510749322


In [174]:
from sklearn.metrics import jaccard_score as jsc
print(jsc(out, resized_test_img, average='micro'))

0.8568983510749322


In [166]:
def dice_coeff(outputs, labels): #F1 score
    mask = 255
    intersections = 0
    h, w = outputs.shape
    for i in range(h):
        for j in range(w):
            if labels[i][j]==mask:
                if labels[i][j]==outputs[i][j]:
                    intersections += 1
                    
    union = len(outputs[outputs>0])+len(labels[labels>0])
    dice = 2*intersections/union
    return dice

In [175]:
print(dice_coeff(resized_test_img, out))

0.922935119823749
